# Behavioural Planning for Autonomous Highway Driving

We plan a trajectory using the _Optimistic Planning for Deterministic systems_ ([OPD](https://hal.inria.fr/hal-00830182)) algorithm.

##  Setup

We first import useful modules for the environment and agent.

In [0]:
# Install environment and agent
!pip install git+https://github.com/eleurent/highway-env#egg=highway-env  > /dev/null 2>&1
!pip install git+https://github.com/eleurent/rl-agents#egg=rl-agents > /dev/null 2>&1

# Environment
import gym
import highway_env

# Agent
from rl_agents.agents.common.factory import agent_factory


In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import Monitor
from pathlib import Path
import base64
from tqdm import tnrange, tqdm

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    html = []
    for mp4 in Path("video").glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


## Run an episode

In [0]:
# Make environment
env = gym.make("highway-v0")
env = Monitor(env, './video', force=True, video_callable=lambda episode: True)
obs, done = env.reset(), False
env.unwrapped.automatic_rendering_callback = env.video_recorder.capture_frame

# Make agent
agent_config = {
    "__class__": "<class 'rl_agents.agents.tree_search.deterministic.DeterministicPlannerAgent'>",
    "env_preprocessors": [{"method":"simplify"}],
    "budget": 40,
    "gamma": 0.7
}
agent = agent_factory(env, agent_config)

# Run episode
steps = tqdm(range(env.unwrapped.config["duration"]), desc="Running")
for _ in steps:
    if done:
      steps.close()
      break
    action = agent.act(obs)
    obs, reward, done, info = env.step(action)
    
env.close()
show_video()